In [53]:
import pandas as pd
import numpy as np

In [54]:
df_initial = pd.read_csv('Zip_Zri_MultiFamilyResidenceRental.csv')
census_df_initial = pd.read_csv('zipcode_census_data_2018_5yr.csv')

In [55]:
include_cities = ['Miami Beach','Miami','North Miami']

df = df_initial.copy()
df = df[df['City'].isin(include_cities)]

census_df = census_df_initial.copy()

In [56]:
# for i in df['City'].unique():
#     if 'miami' in i.lower():
#         print(i)

In [57]:
# monthly_increase = np.mean(df.iloc[:,2:87].diff(axis=1))
# monthly_increase = round(monthly_increase, 2)

In [93]:
df_melt = df.melt(id_vars = ['RegionID','SizeRank','State','Metro','CountyName', 'City','RegionName'], var_name = "date",value_name="rent")
df_melt['year'] = df_melt['date'].apply(lambda x: x.split('-')[0])
df_melt['month'] = df_melt['date'].apply(lambda x: x.split('-')[1])
for col in ['month','year']:
    df_melt[col] = df_melt[col].astype(int)

In [94]:
# for i in df.columns:
#     if '-' in i:
#         print(df[i].unique())

In [95]:
dates = df_melt['date'].unique().tolist()
dates.sort()

In [96]:
len(dates)

113

In [97]:
difference_dict = {}

for i in range(1,len(dates)):
    cur_month = dates[i]
    last_month = dates[i - 1]
    
    test = (df[cur_month]-df[last_month]).mean()
    difference_dict[cur_month] = test

In [98]:
def get_prior_month(date_val):
    month = date_val.split('-')[1]
    year = int(date_val.split('-')[0])
    month_map = {'01':'12',
                 '02':'01',
                 '03':'02',
                 '04':'03',
                 '05':'04',
                 '06':'05',
                 '07':'06',
                 '08':'07',
                 '09':'08',
                 '10':'09',
                 '11':'10',
                 '12':'11'
                }
    
    if month == '01':
        return '%s-%s' % (str(year-1),month_map[month])
    else:
        return '%s-%s' % (str(year),month_map[month])

In [99]:
df_melt.isnull().sum(axis=0)

RegionID        0
SizeRank        0
State           0
Metro           0
CountyName      0
City            0
RegionName      0
date            0
rent          127
year            0
month           0
dtype: int64

In [103]:
df_melt['difference'] = df_melt['date'].apply(lambda x: difference_dict[x] if x in difference_dict else 0)
df_melt['prior_month'] = df_melt['date'].apply(lambda x: get_prior_month(x))

count = 1
maxlim = 100
while count <= maxlim:
    temp_df = df_melt.copy()
    temp_df = temp_df[['RegionID','date','rent']]
    temp_df = temp_df.rename(columns={'date':'prior_month','rent':'prior_month_rent'})
    df_melt = pd.merge(df_melt,temp_df,on=['prior_month','RegionID'],how='left')
    df_melt['rent'] = df_melt['rent'].fillna(df_melt['prior_month_rent']+df_melt['difference'])
    if count != maxlim:
        del df_melt['prior_month_rent']
    count+=1

In [104]:
df_melt.isnull().sum(axis=0)

RegionID              0
SizeRank              0
State                 0
Metro                 0
CountyName            0
City                  0
RegionName            0
date                  0
rent                110
year                  0
month                 0
difference            0
prior_month           0
prior_month_rent    127
dtype: int64